In [1]:
import json
from uuid import uuid4
import requests

In [2]:
from token111 import token
token

'eyJhbGciOiJSUzI1NiIsImtpZCI6IjQwMTU0NmJkMWRhMzA0ZDc2NGNmZWUzYTJhZTVjZDBlNGY2ZjgyN2IiLCJ0eXAiOiJKV1QifQ.eyJuYW1lIjoiQXJ0dXIgS2FiaXJvdiIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS0vQU9oMTRHajRqQVB2aFB6TVhINC0yRnBTb2Z2MDhpaDZ6YVpRRUtOQUJpS2tuUT1zOTYtYyIsImlzcyI6Imh0dHBzOi8vc2VjdXJldG9rZW4uZ29vZ2xlLmNvbS9naWdhdHVybmlwLWI2YjViIiwiYXVkIjoiZ2lnYXR1cm5pcC1iNmI1YiIsImF1dGhfdGltZSI6MTY0MjU3NjA0MSwidXNlcl9pZCI6InpyYlhMdUxwbXBjYWhEbXJnMGtac3o0RlJibTIiLCJzdWIiOiJ6cmJYTHVMcG1wY2FoRG1yZzBrWnN6NEZSYm0yIiwiaWF0IjoxNjQyNTg1NDA1LCJleHAiOjE2NDI1ODkwMDUsImVtYWlsIjoiYXJ0dXJrYWJpcm92MTZAZ21haWwuY29tIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsImZpcmViYXNlIjp7ImlkZW50aXRpZXMiOnsiZ29vZ2xlLmNvbSI6WyIxMTUzMzcxMzAxMzE1MjU0NDAzMjciXSwiZW1haWwiOlsiYXJ0dXJrYWJpcm92MTZAZ21haWwuY29tIl19LCJzaWduX2luX3Byb3ZpZGVyIjoiZ29vZ2xlLmNvbSJ9fQ.WhkxGrtMxtinxCNZlJQ_kQPr7fFlrn-2yYLBNyjZPJcOn-J0bYc0FnZ_tvYmULBqdJkBBYC33cRNHM_r0hcPtp5oNB_4nX2JbKuQqNufWbgoDEHgeb8kWaTALReRg3GQ3KU_-cm_wMW5NnFR8_5dgJzmBoT_g2Z8gCFKh51fZgBcNXUppwqeN

In [3]:
class Requests():
	rest_api = "http://127.0.0.1:8000/api/v1/"
	headers = {
		"Authorization": f"JWT {token}"}
	id = None
	endpoint = None
	data = None

	def post_request(self, url, data):
		response = requests.post(self.rest_api + url, data=data, headers=self.headers)
		print(response.status_code)
		return response

	def get_request(self, url, data=None):
		if not data:
			data = ''
		response = requests.get(self.rest_api + url, data=data, headers=self.headers)
		print(response.status_code)
		return response

	def patch_request(self, url, data):
		response = requests.patch(self.rest_api + url, data=data, headers=self.headers)
		print(response.status_code)
		return response

	def delete_request(self, url):
		response = requests.delete(url)
		print(response.status_code)
		return response

	def create_instance(self):
		self.set_data()
		response = json.loads(self.post_request(self.endpoint, data=self.data).content)
		self.id = response.get('id')
		self.get()

	def update(self):
		self.set_data()
		response = json.loads(self.patch_request(f"{self.endpoint}{self.id}/", data=self.data).content)
		self.id = response.get('id')
		self.get()

	def get(self):
		response = self.get_request(f"{self.endpoint}{self.id}")
		for key in json.loads(response.content):
			setattr(self, key, json.loads(response.content)[key])
		return self

	def get_all(self, params=''):
		response = self.get_request(self.endpoint, data=params)
		return json.loads(response.content)['results']

	def set_id(self, id):
		self.id = id
		self.get()

	def set_data(self):
		data = {}
		for i in list(self.__dict__.items()):
			if i[0] not in ["rest_api", "headers", "endpoint", "data"]:
				i = list(i)
				if i[1].__class__.__module__ != 'builtins':
					i[1] = i[1].id
				data[i[0]] = i[1]
		self.data = data
		return data

In [4]:
class BaseModel():
	def __init__(self, name, description):
		self.name = name
		self.description = description


class Campaign(BaseModel, Requests):

	def __init__(self, name='', description='', managers=[]):
		self.managers = managers
		super().__init__(name, description)
		self.endpoint = 'campaigns/'

	def create_new_chain(self, chain):
		chain.campaign = self
		chain.create_instance()
		return chain


class Chain(BaseModel, Requests):
	def __init__(self, name='', description='', campaign=None):
		self.campaign = campaign
		self.endpoint = "chains/"
		super().__init__(name, description)

	def create_init_stage(self):
		stage = TaskStage(self.id, "Initial Task Stage")
		stage.create_instance()
		return stage


class Stage(Requests):

	def __init__(self, name, desc, chain, in_stages, x=1, y=1):
		self.chain = chain
		self.in_stages = in_stages
		self.name = name
		self.description = desc
		self.x_pos = x
		self.y_pos = y

	def add_stage(self, stage):
		self.get()
		if stage.id not in self.in_stages:
			self.in_stages += [stage.id]

		data = {
			"id": self.id,
			"in_stages": self.in_stages
		}
		response = self.patch_request(f"{self.endpoint}{self.id}/", data)
		if response.status_code != 200:
			self.get()
		return stage


class TaskStage(Stage, Requests):
	def __init__(self, chain=None, name="Task Stage", desc='', in_stages=[], rich_text=None,
				 copy_input=False,
				 allow_multiple_files=False,
				 is_creatable=False,
				 displayed_prev_stages=None,
				 assign_user_by="RA",
				 assign_user_from_stage=None,
				 allow_go_back=False,
				 allow_release=False,
				 is_public=False,
				 webhook_address=None,
				 webhook_payload_field=None,
				 webhook_params=None,
				 webhook_response_field=None):
		self.rich_text = rich_text
		self.copy_input = copy_input
		self.allow_multiple_files = allow_multiple_files
		self.is_creatable = is_creatable
		self.displayed_prev_stages = displayed_prev_stages
		self.assign_user_by = assign_user_by
		self.assign_user_from_stage = assign_user_from_stage
		self.allow_go_back = allow_go_back
		self.allow_release = allow_release
		self.is_public = is_public
		self.webhook_address = webhook_address
		self.webhook_payload_field = webhook_payload_field
		self.webhook_params = webhook_params
		self.webhook_response_field = webhook_response_field
		self.endpoint = "taskstages/"
		super().__init__(name, desc, chain, in_stages)


class ConditionalStage(Stage, Requests):
	def __init__(self, chain, name="Conditional Stage", desc='', in_stages=[], conditions=None, pingpong=False):
		self.conditions = conditions
		self.pingpong = pingpong
		self.endpoint = "conditionalstages/"
		super().__init__(name, desc, chain, in_stages)


class Rank(BaseModel):

	def __init__(self, stages, track=None):
		self.stages = stages
		self.track = track


class RankLimit():

	def __init__(self, rank=Rank, stage=Stage, open_limit=0,
				 total_limit=0,
				 is_listing_allowed=False,
				 is_submission_open=True,
				 is_selection_open=True,
				 is_creation_open=True):
		self.rank = rank
		self.stage = stage
		self.open_limit = open_limit
		self.total_limit = total_limit
		self.is_listing_allowed = is_listing_allowed
		self.is_submission_open = is_submission_open
		self.is_selection_open = is_selection_open
		self.is_creation_open = is_creation_open


class RankRecord():
	def __init__(self, user, rank=None):
		self.user = user
		self.rank = rank


class Task():

	def __init__(self, stage, assignee=None,
				 case=None,
				 responses=None,
				 in_tasks=None,
				 integrator_group=None,
				 complete=False,
				 force_complete=False,
				 reopened=False):
		self.stage = stage
		self.assignee = assignee
		self.case = case
		self.responses = responses
		self.in_tasks = in_tasks
		self.integrator_group = integrator_group
		self.complete = complete
		self.force_complete = force_complete
		self.reopened = reopened


class Track(BaseModel):

	def __init__(self, campaign=Campaign):
		self.campaign = campaign
		self.default_rank = Rank



In [9]:
#After creating campaign u have to add user as a manager
campaign = Campaign(name="Coca cola campaign")
campaign.create_instance()
campaign.__dict__

201
200


{'managers': [],
 'name': 'Coca cola campaign',
 'description': '',
 'endpoint': 'campaigns/',
 'data': {'managers': [], 'name': 'Coca cola campaign', 'description': ''},
 'id': 5,
 'created_at': '2022-01-19T09:48:16.710440Z',
 'updated_at': '2022-01-19T09:48:16.710478Z',
 'open': False,
 'default_track': None}

In [17]:
chain = Chain(name="Monday chain", campaign=campaign)
chain.create_instance()
init_stage = chain.create_init_stage()
init_stage.__dict__

201
200
201
200


In [18]:
next_stage = TaskStage(chain, "Middle task stage. Drink one bottle")
next_stage.create_instance()
next_stage.__dict__

201
200


{'rich_text': None,
 'copy_input': False,
 'allow_multiple_files': False,
 'is_creatable': False,
 'displayed_prev_stages': [],
 'assign_user_by': 'RA',
 'assign_user_from_stage': None,
 'allow_go_back': False,
 'allow_release': False,
 'is_public': False,
 'webhook_address': None,
 'webhook_payload_field': None,
 'webhook_params': None,
 'webhook_response_field': None,
 'endpoint': 'taskstages/',
 'chain': {'id': 9, 'name': 'Monday chain', 'description': '', 'campaign': 6},
 'in_stages': [],
 'name': 'Middle task stage. Drink one bottle',
 'description': '',
 'x_pos': '1.000',
 'y_pos': '1.000',
 'data': {'rich_text': None,
  'copy_input': False,
  'allow_multiple_files': False,
  'is_creatable': False,
  'displayed_prev_stages': None,
  'assign_user_by': 'RA',
  'assign_user_from_stage': None,
  'allow_go_back': False,
  'allow_release': False,
  'is_public': False,
  'webhook_address': None,
  'webhook_payload_field': None,
  'webhook_params': None,
  'webhook_response_field': None,

In [19]:
closing_stage = TaskStage(chain, "Check equipment and pass it to the controller")
closing_stage.create_instance()
closing_stage.__dict__

201
200


{'rich_text': None,
 'copy_input': False,
 'allow_multiple_files': False,
 'is_creatable': False,
 'displayed_prev_stages': [],
 'assign_user_by': 'RA',
 'assign_user_from_stage': None,
 'allow_go_back': False,
 'allow_release': False,
 'is_public': False,
 'webhook_address': None,
 'webhook_payload_field': None,
 'webhook_params': None,
 'webhook_response_field': None,
 'endpoint': 'taskstages/',
 'chain': {'id': 9, 'name': 'Monday chain', 'description': '', 'campaign': 6},
 'in_stages': [],
 'name': 'Check equipment and pass it to the controller',
 'description': '',
 'x_pos': '1.000',
 'y_pos': '1.000',
 'data': {'rich_text': None,
  'copy_input': False,
  'allow_multiple_files': False,
  'is_creatable': False,
  'displayed_prev_stages': None,
  'assign_user_by': 'RA',
  'assign_user_from_stage': None,
  'allow_go_back': False,
  'allow_release': False,
  'is_public': False,
  'webhook_address': None,
  'webhook_payload_field': None,
  'webhook_params': None,
  'webhook_response_fie

In [20]:
init_stage.add_stage(next_stage).add_stage(closing_stage)
init_stage.__dict__

200
200
200
200


{'rich_text': None,
 'copy_input': False,
 'allow_multiple_files': False,
 'is_creatable': False,
 'displayed_prev_stages': [],
 'assign_user_by': 'RA',
 'assign_user_from_stage': None,
 'allow_go_back': False,
 'allow_release': False,
 'is_public': False,
 'webhook_address': None,
 'webhook_payload_field': None,
 'webhook_params': None,
 'webhook_response_field': None,
 'endpoint': 'taskstages/',
 'chain': {'id': 9, 'name': 'Monday chain', 'description': '', 'campaign': 6},
 'in_stages': [24],
 'name': 'Initial Task Stage',
 'description': '',
 'x_pos': '1.000',
 'y_pos': '1.000',
 'data': {'rich_text': None,
  'copy_input': False,
  'allow_multiple_files': False,
  'is_creatable': False,
  'displayed_prev_stages': None,
  'assign_user_by': 'RA',
  'assign_user_from_stage': None,
  'allow_go_back': False,
  'allow_release': False,
  'is_public': False,
  'webhook_address': None,
  'webhook_payload_field': None,
  'webhook_params': None,
  'webhook_response_field': None,
  'chain': 9,


[]